In [5]:
import pandas as pd
import numpy as np
import re

In [6]:
df = pd.read_excel("data/EMEWS original.xlsx", engine='openpyxl', header=[1,2])

/home/kj/dissertation-emews/venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/kj/dissertation-emews/venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


### Cleaning Functions

In [7]:
def print_shape_change(func):
    def wrapper(df, *args, **kwargs):
        shape_before = df.shape
        print(f"[Before] Shape: {shape_before}")
        
        result = func(df, *args, **kwargs)
        
        if result is not None:
            shape_after = result.shape
        else:
            shape_after = df.shape
        print(f"[After]  Shape: {shape_after}")
        print(f"[Change] Rows removed: {shape_before[0] - shape_after[0]}, Columns removed: {shape_before[1] - shape_after[1]}")
        
        return result
    return wrapper

In [8]:
@print_shape_change
def clean_missing_data(df, threshold):
    # Drop columns with missing value ratio above threshold
    missing_ratio = df.isna().mean()
    cols_to_drop = missing_ratio[missing_ratio > threshold].index.tolist()
    print("Dropped columns with > {:.0%} missing values:".format(threshold), cols_to_drop)
    df.drop(columns=cols_to_drop, inplace=True)
    
    # Drop fully empty columns
    empty_cols = df.columns[df.isna().all()].tolist()
    df.drop(columns=empty_cols, inplace=True)

    # Drop rows where all values from column 2 onward are empty
    empty_rows = df.index[df.iloc[:, 2:].isna().all(axis=1)].tolist()
    df.drop(index=empty_rows, inplace=True)


    df.reset_index(drop=True, inplace=True)

In [9]:
@print_shape_change
def remove_total_emews_rows(df):
    first_col = df.columns[0]
    indices_to_remove = df.index[df[first_col].str.lower().str.startswith('total emews')]
    df.drop(index=indices_to_remove, inplace=True)

    df.reset_index(drop=True, inplace=True)

In [10]:
@print_shape_change
def clean_and_flatten_multiindex(df):
    cleaned_cols = []
    for lvl0, lvl1 in df.columns:
        # If level 0 is unnamed or empty, just use level 1
        if 'Unnamed' in str(lvl0) or str(lvl0).strip() == '':
            name = str(lvl1)
        else:
            # Combine both levels
            name = f"{lvl0}_{lvl1}"
        
        # Lowercase, strip spaces, replace spaces with _
        name = name.lower().strip().replace(' ', '_')
        # Replace any -, (, ) with empty string and trailing underscores
        name = name.replace('-', '').replace('(', '').replace(')', '').rstrip('_')
        
        cleaned_cols.append(name)
    
    df.columns = cleaned_cols

### Perform Cleaning

In [11]:
clean_and_flatten_multiindex(df)

[Before] Shape: (747, 35)
[After]  Shape: (747, 35)
[Change] Rows removed: 0, Columns removed: 0


In [12]:
df.columns[df.isna().mean() > 0.95]

Index(['zone_b/c_escalations.1', 'zone_b/c_escalations.2',
       'decision_density__mews_nurse_zone_a_mwr', 'zone_a', 'zone_b/c',
       'total', 'decision_density__cnm_unnamed:_28_level_1',
       'decision_density__cnm_dd_cnm',
       'decision_density__cnm_daily_patient_numbers',
       'decision_density__cnm_skill_mix_senior_both',
       'mandarin_skill_mix_senior', 'white_skill_mix_senior',
       'green_skill_mix_standard'],
      dtype='object')

In [13]:
clean_missing_data(df=df, threshold=0.95)

[Before] Shape: (747, 35)
Dropped columns with > 95% missing values: ['zone_b/c_escalations.1', 'zone_b/c_escalations.2', 'decision_density__mews_nurse_zone_a_mwr', 'zone_a', 'zone_b/c', 'total', 'decision_density__cnm_unnamed:_28_level_1', 'decision_density__cnm_dd_cnm', 'decision_density__cnm_daily_patient_numbers', 'decision_density__cnm_skill_mix_senior_both', 'mandarin_skill_mix_senior', 'white_skill_mix_senior', 'green_skill_mix_standard']
[After]  Shape: (643, 22)
[Change] Rows removed: 104, Columns removed: 13


In [14]:
remove_total_emews_rows(df)

[Before] Shape: (643, 22)
[After]  Shape: (599, 22)
[Change] Rows removed: 44, Columns removed: 0


In [15]:
# Assume each EMEWS set takes 10 mins
# Shift time = 11 hours = 660 minutes

# EMEWS per nurse per day = 660 minutes / 10 minutes per EMEWS = 66

# df["nurses_required_for_emews"] = df["total_number_of_emews"] / 66

In [16]:
df.to_csv('data/EMEWS cleaned.csv', index=False)

### Cleaning Non-Numeric and Whitespace-Only Values

In [17]:
# Identify columns containing empty/whitespace-only strings
cols_with_missing = [
    col for col in df.columns
    if df[col].astype(str).str.match(r'^\s*$').any()
]

for col in cols_with_missing:
    df[col] = pd.to_numeric(df[col], errors='coerce')

still_non_numeric_cols = set(df.columns) - set(df.select_dtypes(include='number').columns) - {'date', 'day'}

In [18]:
for col in still_non_numeric_cols:
    non_numeric_values = df[col][pd.to_numeric(df[col], errors='coerce').isna()]
    print(f"Non-numeric values in column '{col}': {non_numeric_values.unique()}")

    replace_map = {}

    for val in list(non_numeric_values.unique()):
        # Use regex to check if string starts with digits
        if pd.notna(val):
            match = re.match(r'^(\d+)', val)
            if match:
                replace_map[val] = match.group(1)
            else:
                replace_map[val] = np.nan

    print(f"Replace map: {replace_map}", end='\n\n')
    df[col] = df[col].replace(replace_map)
    df[col] = pd.to_numeric(df[col], errors='coerce')

Non-numeric values in column 'zone_a__sets_of_emews': [nan '81 - stopped @6pm' '68 -commenced 12md']
Replace map: {'81 - stopped @6pm': '81', '68 -commenced 12md': '68'}

Non-numeric values in column 'zone_a_mwr_deescalations': [nan 'Nurse redeployed' 'Nurse Re-deployed' 'stopped at 2']
Replace map: {'Nurse redeployed': nan, 'Nurse Re-deployed': nan, 'stopped at 2': nan}

Non-numeric values in column 'zone_a_mwr_escalations': [nan '0 - stopped 16.00' '2 - commenced 15.00']
Replace map: {'0 - stopped 16.00': '0', '2 - commenced 15.00': '2'}

Non-numeric values in column 'zone_a__cat_3': [nan]
Replace map: {}

Non-numeric values in column 'zone_b/c_sets_of_emews': [nan '7 *commenced @ 14.00' '43 *stopped 16.40' '51 *commenced 10.44am'
 '64 - paused for 2hrs']
Replace map: {'7 *commenced @ 14.00': '7', '43 *stopped 16.40': '43', '51 *commenced 10.44am': '51', '64 - paused for 2hrs': '64'}



In [19]:
print(f"Total Number of Columns in Dataframe: {len(df.columns)}")
print(f"Numeric Columns in Dataframe(should be total - 2(date cols):{len(df.select_dtypes(include='number').columns)}")

Total Number of Columns in Dataframe: 22
Numeric Columns in Dataframe(should be total - 2(date cols):20


### Adjusting Day and Date Columns to Handle AM/PM Time Information for Duplicate Dates

In [20]:
df['date'] = pd.to_datetime(df['date'])

df['day'] = df['day'].str.lower()
original_values = df['day'].copy()
df['day'] = df['day'].str.strip()
num_cells_stripped = ((original_values != df['day']) & original_values.notna()).sum()

print(f"Whitespace removed from {num_cells_stripped} cells in column 'day'.")

Whitespace removed from 51 cells in column 'day'.


In [21]:
df['am_pm'] = df['day'].str.extract(r'(am|pm)$', expand=False).fillna('am')
dup_dates = df['date'][df['date'].duplicated(keep=False)]

print(f"Count of duplicated dates: {dup_dates.count()}")
print(f"Starting date: {dup_dates.iloc[0]}, Starting Index: {dup_dates.index[0]}")
print(f"Rows == dup_dates count + start idx --> {df.shape[0] == dup_dates.count() + dup_dates.index[0]}")

Count of duplicated dates: 568
Starting date: 2024-01-22 00:00:00, Starting Index: 17
Rows == dup_dates count + start idx --> False


In [22]:
indices = dup_dates.index.to_list()

# Find gaps where continuity breaks
break_points = [(indices[i], indices[i+1]) for i in range(len(indices)-1) if indices[i+1] != indices[i] + 1]

print("Continuity breaks between these index pairs:")
for start_idx, end_idx in break_points:
    print(f"Gap between {start_idx} and {end_idx}")

Continuity breaks between these index pairs:
Gap between 256 and 261
Gap between 270 and 275
Gap between 348 and 351
Gap between 396 and 399
Gap between 496 and 499


In [23]:
print("\nValues in the gap ranges:", end='\n\n')

for start_idx, end_idx in break_points:
    # The gap range is indices between start_idx+1 and end_idx-1 (exclusive)
    gap_range = range(start_idx + 1, end_idx)
    values_in_gap = df.iloc[gap_range]['date']
    print(f"Values between indices {start_idx+1} and {end_idx-1}:")
    print(values_in_gap)
    print("-" * 40)


Values in the gap ranges:

Values between indices 257 and 260:
257   2024-05-21
258   2014-05-21
259   2014-05-22
260   2024-05-22
Name: date, dtype: datetime64[ns]
----------------------------------------
Values between indices 271 and 274:
271   2024-05-28
272   2014-05-28
273   2014-05-29
274   2024-05-29
Name: date, dtype: datetime64[ns]
----------------------------------------
Values between indices 349 and 350:
349   2014-07-06
350   2024-07-06
Name: date, dtype: datetime64[ns]
----------------------------------------
Values between indices 397 and 398:
397   2024-07-30
398   2014-07-30
Name: date, dtype: datetime64[ns]
----------------------------------------
Values between indices 497 and 498:
497   2024-09-18
498   2014-09-18
Name: date, dtype: datetime64[ns]
----------------------------------------


In [24]:
# Update year from 2014 to 2024 in 'date' column
mask_2014 = df['date'].dt.year == 2014
df.loc[mask_2014, 'date'] = df.loc[mask_2014, 'date'].apply(lambda dt: dt.replace(year=2024))

dup_dates = df['date'][df['date'].duplicated(keep=False)]
indices = dup_dates.index.to_list()

# Check if duplicated date indices are continuous
are_continuous = all(b == a + 1 for a, b in zip(indices, indices[1:]))
print(f"Are all indices continuous --> {are_continuous}")

print(f"Count of duplicated dates: {dup_dates.count()}")
print(f"Starting date: {dup_dates.iloc[0]}, Starting Index: {dup_dates.index[0]}")
print(f"Rows == dup_dates count + start idx --> {df.shape[0] == dup_dates.count() + dup_dates.index[0]}")

Are all indices continuous --> True
Count of duplicated dates: 582
Starting date: 2024-01-22 00:00:00, Starting Index: 17
Rows == dup_dates count + start idx --> True


In [25]:
mask_dup = df['date'].isin(dup_dates)

# Remove 'am' or 'pm' suffix from 'day' column only for duplicated date rows
df.loc[mask_dup, 'day'] = df.loc[mask_dup, 'day'].str.replace(r'\s*(am|pm)$', '', regex=True)

# For duplicated date rows with 'pm', add 12 hours to 'date' to represent afternoon time
df.loc[mask_dup & (df['am_pm'] == 'pm'), 'date'] += pd.Timedelta(hours=12)

df.drop(columns=['am_pm'], inplace=True)

In [26]:
df['day'].unique()

array(['thursday', 'friday', 'saturday', 'sunday', 'monday', 'tuesday',
       'wednesday', 'wenesday', 'wedneday', 'tue'], dtype=object)

In [27]:
fix_map = {
    'tue': 'tuesday',
    'wenesday': 'wednesday',
    'wedneday': 'wednesday'
}

df['day'] = df['day'].replace(fix_map)
df['day'].unique()

array(['thursday', 'friday', 'saturday', 'sunday', 'monday', 'tuesday',
       'wednesday'], dtype=object)

### Save the cleaned file

In [28]:
df.isna().mean() * 100

day                          0.000000
date                         0.000000
total_number_of_patients     0.333890
total_number_of_emews        0.333890
zone_a_mwr_patients         42.904841
zone_a_mwr_cat_3            43.071786
zone_a_mwr_cat_4            43.071786
zone_a_mwr_sets_of_emews    45.409015
zone_a_mwr_deescalations    43.739566
zone_a_mwr_escalations      43.405676
zone_a__patients             2.671119
zone_a__cat_2                2.671119
zone_a__cat_3                2.671119
zone_a__sets_of_emews        2.671119
zone_a__deescalations        2.838063
zone_a__escalations          2.838063
zone_b/c_patients           86.477462
zone_b/c_cat_2              87.312187
zone_b/c_cat_3              87.312187
zone_b/c_sets_of_emews      86.477462
zone_b/c_deescalations      86.644407
zone_b/c_escalations        86.644407
dtype: float64

In [29]:
df.to_csv("data/EMEWS_cleaned_with_nan.csv", index=False)

In [30]:
def infer_shift_duration(current, next_time):
    if pd.isnull(next_time):
        return 12  # handle last row
    delta = (next_time - current).total_seconds() / 3600
    if delta <= 13:
        return 12
    else:
        return 24

In [31]:
AVG_EMEWS_TIME = 8
df.fillna(0, inplace=True)

In [32]:
df['next_date'] = df['date'].shift(-1)
df['shift_hours'] = df.apply(lambda row: infer_shift_duration(row['date'], row['next_date']), axis=1)


# for each patient, they are expected to receive the below number of EMEWS sets per shift based on their triage category
screenings_per_shift = {
    2: {12: 72, 24: 144},  # every 10 mins
    3: {12: 12, 24: 24},   # every 1 hr
    4: {12: 6,  24: 12}    # every 2 hrs
}

In [33]:
def nurse_minutes_per_category(row):
    shift = row['shift_hours']
    return pd.Series({
        'cat2_minutes': (
            row.get('zone_a__cat_2', 0) + row.get('zone_b/c_cat_2', 0)
        ) * screenings_per_shift[2][shift] * 8,
        
        'cat3_minutes': (
            row.get('zone_a__cat_3', 0) +
            row.get('zone_a_mwr_cat_3', 0) +
            row.get('zone_b/c_cat_3', 0)
        ) * screenings_per_shift[3][shift] * 8,

        'cat4_minutes': (
            row.get('zone_a_mwr_cat_4', 0)
        ) * screenings_per_shift[4][shift] * 8,
    })

cat_minutes_df = df.apply(nurse_minutes_per_category, axis=1)

df = pd.concat([df, cat_minutes_df], axis=1)

df['cat2_nurses'] = round(df['cat2_minutes'] / 60 / df['shift_hours'])
df['cat3_nurses'] = round(df['cat3_minutes'] / 60 / df['shift_hours'])
df['cat4_nurses'] = round(df['cat4_minutes'] / 60 / df['shift_hours'])

df['total_nurses'] = df[['cat2_nurses', 'cat3_nurses', 'cat4_nurses']].sum(axis=1)
df.drop(columns=['cat2_minutes','cat3_minutes', 'cat4_minutes', 'next_date', 'shift_hours'])

,day,date,total_number_of_patients,total_number_of_emews,zone_a_mwr_patients,zone_a_mwr_cat_3,zone_a_mwr_cat_4,zone_a_mwr_sets_of_emews,zone_a_mwr_deescalations,zone_a_mwr_escalations,...,zone_b/c_patients,zone_b/c_cat_2,zone_b/c_cat_3,zone_b/c_sets_of_emews,zone_b/c_deescalations,zone_b/c_escalations,cat2_nurses,cat3_nurses,cat4_nurses,total_nurses
0,thursday,2024-01-04 00:00:00,100.0,302.0,64.0,53.0,11.0,192.0,10.0,6.0,...,15.0,0.0,0.0,60.0,1.0,5.0,10.0,8.0,1.0,19.0
1,friday,2024-01-05 00:00:00,112.0,220.0,59.0,59.0,0.0,90.0,10.0,0.0,...,27.0,12.0,15.0,70.0,0.0,5.0,21.0,11.0,0.0,32.0
2,saturday,2024-01-06 00:00:00,69.0,175.0,36.0,36.0,0.0,58.0,3.0,2.0,...,15.0,0.0,0.0,50.0,0.0,5.0,14.0,5.0,0.0,19.0
3,sunday,2024-01-07 00:00:00,74.0,206.0,30.0,27.0,3.0,41.0,2.0,11.0,...,16.0,0.0,0.0,50.0,0.0,5.0,14.0,4.0,0.0,18.0
4,monday,2024-01-08 00:00:00,87.0,267.0,41.0,38.0,3.0,116.0,10.0,1.0,...,23.0,10.0,13.0,78.0,0.0,4.0,22.0,7.0,0.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,tuesday,2024-11-05 12:00:00,55.0,197.0,27.0,27.0,0.0,73.0,25.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,4.0,0.0,24.0
595,wednesday,2024-11-06 00:00:00,50.0,235.0,26.0,26.0,0.0,0.0,26.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18.0,4.0,0.0,22.0
596,wednesday,2024-11-06 12:00:00,63.0,196.0,36.0,36.0,0.0,83.0,15.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,17.0,6.0,0.0,23.0
597,thursday,2024-11-07 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,4.0,0.0,15.0


In [34]:
df

,day,date,total_number_of_patients,total_number_of_emews,zone_a_mwr_patients,zone_a_mwr_cat_3,zone_a_mwr_cat_4,zone_a_mwr_sets_of_emews,zone_a_mwr_deescalations,zone_a_mwr_escalations,...,zone_b/c_escalations,next_date,shift_hours,cat2_minutes,cat3_minutes,cat4_minutes,cat2_nurses,cat3_nurses,cat4_nurses,total_nurses
0,thursday,2024-01-04 00:00:00,100.0,302.0,64.0,53.0,11.0,192.0,10.0,6.0,...,5.0,2024-01-05 00:00:00,24,13824.0,11904.0,1056.0,10.0,8.0,1.0,19.0
1,friday,2024-01-05 00:00:00,112.0,220.0,59.0,59.0,0.0,90.0,10.0,0.0,...,5.0,2024-01-06 00:00:00,24,29952.0,16512.0,0.0,21.0,11.0,0.0,32.0
2,saturday,2024-01-06 00:00:00,69.0,175.0,36.0,36.0,0.0,58.0,3.0,2.0,...,5.0,2024-01-07 00:00:00,24,20736.0,6912.0,0.0,14.0,5.0,0.0,19.0
3,sunday,2024-01-07 00:00:00,74.0,206.0,30.0,27.0,3.0,41.0,2.0,11.0,...,5.0,2024-01-08 00:00:00,24,19584.0,6336.0,288.0,14.0,4.0,0.0,18.0
4,monday,2024-01-08 00:00:00,87.0,267.0,41.0,38.0,3.0,116.0,10.0,1.0,...,4.0,2024-01-09 00:00:00,24,32256.0,10752.0,288.0,22.0,7.0,0.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,tuesday,2024-11-05 12:00:00,55.0,197.0,27.0,27.0,0.0,73.0,25.0,0.0,...,0.0,2024-11-06 00:00:00,12,14400.0,2880.0,0.0,20.0,4.0,0.0,24.0
595,wednesday,2024-11-06 00:00:00,50.0,235.0,26.0,26.0,0.0,0.0,26.0,0.0,...,0.0,2024-11-06 12:00:00,12,13248.0,2592.0,0.0,18.0,4.0,0.0,22.0
596,wednesday,2024-11-06 12:00:00,63.0,196.0,36.0,36.0,0.0,83.0,15.0,2.0,...,0.0,2024-11-07 00:00:00,12,12096.0,4032.0,0.0,17.0,6.0,0.0,23.0
597,thursday,2024-11-07 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2024-11-07 12:00:00,12,8064.0,2688.0,0.0,11.0,4.0,0.0,15.0


In [35]:
df.to_csv("data/EMEWS_cleaned_with_fillna_0.csv", index=False)